In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from autoreduce import *
import numpy as np
from sympy import symbols

# Post conservation law and other approximations phenomenological model at the RNA level
n = 8 # Number of states : P, C1, T, R, C2, E, C3, X
nouts = 1 # Number of outputs, X_i

# Inputs by user 
x_init = np.zeros(n)
x_init[0] = 100
x_init[3] = 400
x_init[5] = 20
C = np.zeros((nouts,n), dtype=int)
C[0][7] = 1

nstates_tol_max = 3
nsatees_tol_min = 2
error_tol = 3000
# System dynamics symbolically

# k_bp, k_up, k_tx, k_br, k_ur, k_tl, k_be, k_ue, d_i, d = params, len(params) = 10


x0 = symbols('P')
x1 = symbols('C1') # G:P
x2 = symbols('T')
x3 = symbols('R')
x4 = symbols('C2') # T:R
x5 = symbols('E')
x6 = symbols('C3') # T:E
x7 = symbols('X')

x = [x0, x1, x2, x3, x4, x5, x6, x7]

G = symbols('G')
k_bp = symbols('k_bp')
k_up = symbols('k_up')
k_tx = symbols('k_tx')
k_br = symbols('k_br')
k_ur = symbols('k_ur')
k_tl = symbols('k_tl')
k_be = symbols('k_be')
k_ue = symbols('k_ue')
d_i = symbols('d_i')
d = symbols('d')

E_tot = symbols('E_tot')
P_tot = symbols('P_tot')
R_tot = symbols('R_tot')
params = [k_bp, k_up, k_tx, k_br, k_ur, k_tl, k_be, k_ue, d_i, d, E_tot, P_tot, R_tot, G]
# f0 = (k_bp + k_tx) * x1 - k_up * G * x0
f0 = (k_up + k_tx) * x1 - k_bp * G * x0
f1 = k_bp * G * x0 - (k_up + k_tx)*x1
f2 = k_tx * x1 + (k_ur + k_tl) * x4 - k_br * x2 * x3
f3 = (k_ur + k_tl) * x4 - k_br * x2 * x3
f4 = k_br * x2 * x3 - (k_ur + k_tl) * x4
f5 = (k_ue + d_i) * x6 - k_be * x2 * x5
f6 = k_be * x2 * x5 - (k_ue + d_i) * x6
f7 = k_tl * x4 - d * x7
    
f = [f0,f1,f2,f3,f4,f5,f6,f7]
# parameter values
# E_tot = 20
# P_tot = 100
# R_tot = 400
params_values = [30, 10, 0.50, 80, 2, 8, 10, 2, 1, 0.5, 20, 100, 400, 10]
# params_values = [100, 10, 4, 10, 0.25, 2, 10, 0.5, 1, 1, 1000, 1000, 1000, 10]
sys = System(x, f, params = params, params_values = params_values, C = C, x_init = x_init)

In [2]:
from autoreduce.utils import get_reducible
timepoints_ssm = np.linspace(0,2,10)
timepoints_ode = np.linspace(0,2,100)
sys_reduce = get_reducible(sys, timepoints_ode, timepoints_ssm)
sys_reduce.nstates_tol_min = 2
sys_reduce.nstates_tol_max = 3

In [3]:
P, C1, T, R, C2, E, C3, X = sys.x
conserved_quantities = [P + C1 - P_tot, R + C2 - R_tot, E + C3 - E_tot]
states_to_eliminate = [C1, C2, C3]
f_cons = sys_reduce.set_conservation_laws(conserved_quantities, states_to_eliminate)

In [4]:
sys_reduce.f

[-G*P*k_bp + (-P + P_tot)*(k_tx + k_up),
 -R*T*k_br + k_tx*(-P + P_tot) + (-R + R_tot)*(k_tl + k_ur),
 -R*T*k_br + (-R + R_tot)*(k_tl + k_ur),
 -E*T*k_be + (-E + E_tot)*(d_i + k_ue),
 -X*d + k_tl*(-R + R_tot)]

In [5]:
reduced_sys, fast_ss = sys_reduce.solve_timescale_separation([T,X], debug = True)


Reduced set of variables is [T, X]
f_hat =  [-R*T*k_br + k_tx*(-P + P_tot) + (-R + R_tot)*(k_tl + k_ur), -X*d + k_tl*(-R + R_tot)]
Collapsed set of variables is [P, R, E]
In sympy_solve_and_substitute, solving for  P
From  -G*P*k_bp + (-P + P_tot)*(k_tx + k_up)
Solution found:  {P: [P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up)]}
Updated f_hat now is  [-R*T*k_br + k_tx*(-P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up) + P_tot) + (-R + R_tot)*(k_tl + k_ur), -X*d + k_tl*(-R + R_tot)]
In sympy_solve_and_substitute, solving for  R
From  -R*T*k_br + (-R + R_tot)*(k_tl + k_ur)
Solution found:  {P: [P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up)], R: [R_tot*(k_tl + k_ur)/(T*k_br + k_tl + k_ur)]}
Updated f_hat now is  [-R_tot*T*k_br*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + k_tx*(-P_tot*(k_tx + k_up)/(G*k_bp + k_tx + k_up) + P_tot) + (k_tl + k_ur)*(-R_tot*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + R_tot), -X*d + k_tl*(-R_tot*(k_tl + k_ur)/(T*k_br + k_tl + k_ur) + R_tot)]
In sympy_solve_and_substitute, solving fo